In [282]:
# import nltk
# import streamlit as st
# import pandas as pd
# import spacy
# import pyinflect
# import random     
# import numpy as np



#from create_exercise import Create_exercise

# nlp = spacy.load("en_core_web_sm") 

In [303]:
pip freeze

absl-py==1.4.0
aiohttp==3.8.4
aiosignal==1.3.1
alabaster @ file:///home/conda/feedstock_root/build_artifacts/alabaster_1673645646525/work
altair==5.0.1
amqp==5.1.1
anaconda-client @ file:///home/conda/feedstock_root/build_artifacts/anaconda-client_1668463703032/work
anaconda-navigator==2.3.2
anyascii==0.3.2
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1666191106763/work/dist
appdirs @ file:///home/conda/feedstock_root/build_artifacts/appdirs_1603108395799/work
applaunchservices @ file:///Users/runner/miniforge3/conda-bld/applaunchservices_1653840691220/work
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1649077682618/work
appscript @ file:///Users/runner/miniforge3/conda-bld/appscript_1667548357055/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1666851004481/work
arrow @ file:///home/conda/feedstock_root/build

In [283]:
import nltk
import streamlit as st
import pandas as pd
import spacy
import pyinflect
import random     
import numpy as np
from ast import literal_eval

from tqdm import tqdm

nlp = spacy.load("en_core_web_sm") 

# Главный класс приложения
class Create_exercise:
    def __init__(self, task='', answer=''):
        #self.df = df
        #self.options=options
        self.task = task
        self.answer = answer
        #self.exercise_type = exercise_type
        

 # Загрузка текста 
    #@st.cache_data
    def get_text(self):
        with open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt') as f:
            self.text = f.read()
            self.text = self.text.replace('\n','')
            return self.text
        
# Приводим короткие формы глаголов к полной
    def contracted_text(self, text): 
        expanded_words = []   
        for word in text.split():
            expanded_words.append(contractions.fix(word))  
        expanded_text = ' '.join(expanded_words)    
        return expanded_text
        
# Очистка текста        
   # @st.cache_data
    def clear_text(self, text):
        self.text = self.text.lower()
        self.text = self.text.replace('"', '')
        self.text = self.text.replace(',', '')
        self.text = self.text.replace(':', '')
        self.text = self.text.replace('-"', '')
        return self.text

    #st.cache_data
    def create_df(self):
        self.df = pd.DataFrame(columns=['sentence', 'options', 'answer', 'task', 'result'])
        # self.text = text
        # tokens_sens = nltk.tokenize.sent_tokenize(self.text, language='english')
        # df_sentences = pd.DataFrame({'sentence': tokens_sens})
        # df_sentences["sentence"]= df_sentences.apply(lambda x: x['sentence'].replace('.', ''), axis=1)
        return self.df

# Токенизация по предложениям и создание ДФ
    #@st.cache_data
    def tokenization(self, text):
        self.text=text
        self.tokens_sens = nltk.tokenize.sent_tokenize(self.text, language='english')
        self.df_sentences = pd.DataFrame({'sentence': self.tokens_sens})
        self.df_sentences["sentence"]= self.df_sentences.apply(lambda x: x['sentence'].replace('.', ''), axis=1)    
        return self.df_sentences

# Обработка упражнения и вывод итогового ДФ
    #@st.cache_data
    def select_exercise(self, df_sentences, exercise_type):
        options = []
        print('exercise_type=', exercise_type)
        
        try:
            self.df = pd.read_csv('df_VERB.csv', converters={'options': literal_eval})
            print('Загрузили фрейм')
            self.df.drop(columns=['result'], axis=1, inplace=True)
            display(self.df)
            
            print(option)
            self.df.options = self.df.options.astype(list)
            return self.df
        except:
            print("Сработало исключение")
            print("Код работает дальше")
            for sentence in df_sentences.sentence:
                for token in nlp(str(sentence)):
                    if token.pos_=='VERB' and exercise_type == 'Выберите правильную форму глагола':                    
                        self.answer = [token.text for token in nlp(str(sentence)) if token.pos_=='VERB']
                        options.append(list(set([token._.inflect('VB'), token._.inflect('VBN'), token._.inflect('VBP'), token._.inflect('VBZ'), token._.inflect('VBG'), token._.inflect('VBD')])))
                        self.task = token.pos_
                        write_it_df=1
                    elif token.pos_=='ADJ'and exercise_type == 'Выбор правильного прилагательного':
                        self.answer = [token.text for token in nlp(str(sentence)) if token.pos_=='ADJ']
                        options.append([token.text, token._.inflect('JJS')])
                        self.task = token.pos_
                        write_it_df=1
                
                    elif exercise_type ==  'Расставьте в правильном порядке слова предложения'  and len(nlp(str(sentence))) < 9:
                        options = [token.text for token in nlp(str(sentence))]
                        options = [options] * len(options)
                        self.answer = [token.text for token in nlp(str(sentence))]
                            
                        write_it_df=1
                        self.task = 'order_words'
                    else: pass                

                if exercise_type == 'Выберите правильный артикль':
                    self.task = 'articles'
                    self.answer=[]
                    split_string = sentence.split(" ")
                    for i in split_string:
                        for j in ['a', 'the', 'an']:
                            if i==j:
                                self.answer.append(i)
                                options.append([' a ', ' the ', ' an '])
                                break  
                    self.answer = list(map(lambda x: ' '+ x + ' ', self.answer))  
                    write_it_df=1     
                #display(self.df)
                if len(nlp(str(sentence))) in range(3, 20) and len(self.answer) > 0 and write_it_df==1:                    
                    self.df.loc[len(self.df)]=[sentence, options, self.answer, self.task, []]  

                # сбрасываем переменные    
                options=[]  
                write_it_df=0    
                self.answer=[]        

            self.df["sentence_hidden"] = self.df["sentence"]
            for index, row in self.df.iterrows(): 
                for i in row.answer:
                    if exercise_type == 'Расставьте в правильном порядке слова предложения':
                        self.df["sentence_hidden"][index] =  '__ ' * len(row.answer)
                    else: self.df["sentence_hidden"][index] = self.df["sentence_hidden"][index].replace(i, ' ___ ')
            filename = f'df_{self.task}.csv'
            self.df.to_csv(filename, index=False)        
                        
            return self.df

In [284]:
nltk.download('punkt')

random.seed(42)

[nltk_data] Downloading package punkt to /Users/Sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [285]:
exercise_type = 'Выберите правильную форму глагола'   
create_exercise = Create_exercise()   

text = st.text_area('Текст', create_exercise.get_text())

#text = create_exercise.contracted_text(text)

#exercise_type = st.sidebar.selectbox('Выберите тип упражнения:', ['', 'Выберите правильную форму глагола',  'Выбор правильного прилагательного', 'Выберите правильный артикль', 'Расставьте в правильном порядке слова предложения'], format_func=lambda x: 'Ничего не выбрано' if x == '' else x)
  

In [286]:
num_of_sentenses = 5

text = create_exercise.clear_text(text)

In [287]:
options = []

df = create_exercise.create_df()

df_sentences = create_exercise.tokenization(text)    
df = create_exercise.select_exercise(df_sentences, exercise_type)

key=0
num_of_sentenses = 5
df = df[0:num_of_sentenses]

exercise_type= Выберите правильную форму глагола
Загрузили фрейм
Сработало исключение
Код работает дальше


In [288]:
#options
df

,sentence,options,answer,task,sentence_hidden
0,once upon a time there was a sweet little girl,"[[been, being, be, am, was, is]]",['was'],VERB,once upon ___ time there ___ ___ ___ __...
1,once she gave her a little cap made of red velvet,"[[given, gives, gave, giving, give], [make, ma...","['gave', 'made']",VERB,onc ___ ___ ___ ___ sh ___ ___ ___ ___ ___ ...
2,take them to your grandmother,"[[take, taking, takes, taken, took]]",['take'],VERB,___ ___ ___ ___ ___ h ___ m ___ o your ...
3,she is sick and weak and they will do her well,"[[do, did, done, doing, does]]",['do'],VERB,she is sick an ___ weak an ___ they will __...
4,mind your manners and give her my greetings,"[[mind, minds, minded, minding], [given, gives...","['mind', 'give']",VERB,___ ___ ___ ___ ___ ___ ___ ___ ___ ___ __...


In [289]:
exercise_type = 'Расставьте в правильном порядке слова предложения'   
exercise_type = 'Расставьте в правильном порядке слова предложения'   
exercise_type = 'Расставьте в правильном порядке слова предложения'   
#exercise_type = 'Выберите правильную форму глагола'   

In [290]:
pd.read_csv('df_VERB_old.csv').head(5)

,sentence,options,answer,task,result,sentence_hidden
0,once upon a time there was a sweet little girl,"[['is', 'be', 'been', 'am', 'was', 'being']]",['was'],VERB,[],once upon a time there ___ a sweet little girl
1,once she gave her a little cap made of red velvet,"[['giving', 'gives', 'gave', 'given', 'give'],...","['gave', 'made']",VERB,[],once she ___ her a little cap ___ of red v...
2,take them to your grandmother,"[['taking', 'take', 'taken', 'takes', 'took']]",['take'],VERB,[],___ them to your grandmother
3,she is sick and weak and they will do her well,"[['did', 'does', 'doing', 'do', 'done']]",['do'],VERB,[],she is sick and weak and they will ___ her well
4,mind your manners and give her my greetings,"[['minding', 'minded', 'mind', 'minds'], ['giv...","['mind', 'give']",VERB,[],___ your manners and ___ her my greetings


In [291]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sentence         5 non-null      object
 1   options          5 non-null      object
 2   answer           5 non-null      object
 3   task             5 non-null      object
 4   sentence_hidden  5 non-null      object
dtypes: object(5)
memory usage: 240.0+ bytes


In [292]:
df["result"] = np.nan
df

/var/folders/3s/0vl43zsj2n79hlj09xb7sby40000gn/T/ipykernel_2703/1137173519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["result"] = np.nan


,sentence,options,answer,task,sentence_hidden,result
0,once upon a time there was a sweet little girl,"[[been, being, be, am, was, is]]",['was'],VERB,once upon ___ time there ___ ___ ___ __...,NaN
1,once she gave her a little cap made of red velvet,"[[given, gives, gave, giving, give], [make, ma...","['gave', 'made']",VERB,onc ___ ___ ___ ___ sh ___ ___ ___ ___ ___ ...,NaN
2,take them to your grandmother,"[[take, taking, takes, taken, took]]",['take'],VERB,___ ___ ___ ___ ___ h ___ m ___ o your ...,NaN
3,she is sick and weak and they will do her well,"[[do, did, done, doing, does]]",['do'],VERB,she is sick an ___ weak an ___ they will __...,NaN
4,mind your manners and give her my greetings,"[[mind, minds, minded, minding], [given, gives...","['mind', 'give']",VERB,___ ___ ___ ___ ___ ___ ___ ___ ___ ___ __...,NaN


In [300]:






pd.set_option('display.max_rows', 20) 
# print(row['options'].dtype())
# print(row['options'][0])
# print(len['options'])
# print(len(row['options']))
for index, row in df.iterrows():
    print(row['sentence_hidden'])
    print(row['sentence'])
    
    option = []
    for i in range(len(row['options'])):
        print(len(row['options']))
        option = row['options'][i]
        random.shuffle(option)
        option = ['–––'] + option
        #option.append(['–––'])
        #df['result'][index] =  st.selectbox('nolabel', option, label_visibility="hidden", key =str(key) )
        df['result'][index] =  'any'

        if df['result'][index] == '–––':
            pass

        elif df['result'][index] == str(row['answer'][i]):
            print('Это правльный ответ')
        
        else:
            print('Попробуйте еще раз')
    print('----------------------')       
print(exercise_type)         

once upon  ___  time there  ___  ___  ___   ___   ___  ___ eet little girl
once upon a time there was a sweet little girl
1
Попробуйте еще раз
----------------------
onc ___ ___ ___  ___ sh ___ ___ ___  ___  ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ h ___ ___ ___ r ___  ___ ___ ___  ___ littl ___ ___ ___  ___ c ___ ___ ___ p ___  ___  ___ ___ ___  ___  ___ ___ ___  ___ of ___ r ___ ___ ___  ___  ___  ___ ___ ___  ___ ___ ___ l ___ ___ ___  ___ ___ ___ t
once she gave her a little cap made of red velvet
2
Попробуйте еще раз
2
Попробуйте еще раз
----------------------
 ___  ___  ___  ___   ___ h ___ m  ___ o your gr ___ ndmo ___ h ___ r
take them to your grandmother
1
Попробуйте еще раз
----------------------
she is sick an ___  weak an ___  they will  ___  ___  her well
she is sick and weak and they will do her well
1
Попробуйте еще раз
----------------------
 ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ your ___  ___ ___ ___ a ___ ___ ___  ___ ___ ___  ___ rs _

/var/folders/3s/0vl43zsj2n79hlj09xb7sby40000gn/T/ipykernel_2703/2471159057.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result'][index] =  'any'


In [294]:
df

,sentence,options,answer,task,sentence_hidden,result
0,once upon a time there was a sweet little girl,"[[am, being, be, was, been, is]]",['was'],VERB,once upon ___ time there ___ ___ ___ __...,any
1,once she gave her a little cap made of red velvet,"[[giving, gave, given, give, gives], [makes, m...","['gave', 'made']",VERB,onc ___ ___ ___ ___ sh ___ ___ ___ ___ ___ ...,any
2,take them to your grandmother,"[[taking, takes, taken, took, take]]",['take'],VERB,___ ___ ___ ___ ___ h ___ m ___ o your ...,any
3,she is sick and weak and they will do her well,"[[does, doing, done, do, did]]",['do'],VERB,she is sick an ___ weak an ___ they will __...,any
4,mind your manners and give her my greetings,"[[minded, minds, mind, minding], [gives, givin...","['mind', 'give']",VERB,___ ___ ___ ___ ___ ___ ___ ___ ___ ___ __...,any


In [295]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sentence         5 non-null      object
 1   options          5 non-null      object
 2   answer           5 non-null      object
 3   task             5 non-null      object
 4   sentence_hidden  5 non-null      object
 5   result           5 non-null      object
dtypes: object(6)
memory usage: 452.0+ bytes


In [296]:
for i in df.options:
    print(type(i))
    for j in i:
        print(j)

<class 'list'>
['am', 'being', 'be', 'was', 'been', 'is']
<class 'list'>
['giving', 'gave', 'given', 'give', 'gives']
['makes', 'making', 'made', 'make']
<class 'list'>
['taking', 'takes', 'taken', 'took', 'take']
<class 'list'>
['does', 'doing', 'done', 'do', 'did']
<class 'list'>
['minded', 'minds', 'mind', 'minding']
['gives', 'giving', 'given', 'gave', 'give']


In [297]:
pd.set_option('display.max_rows', 20) 
# print(row['options'].dtype())
# print(row['options'][0])
# print(len['options'])
# print(len(row['options']))
for index, row in df.iterrows():
    print(str(row['sentence_hidden']))
    print(str(row['sentence']))
    
    option = []
    for i in range(len(list(row['options']))):
        print(i)
        option = list(row['options'][i])
        random.shuffle(option)
        option = ['–––'] + option
        #option.append(['–––'])
        #df['result'][index] =  st.selectbox('nolabel', option, label_visibility="hidden", key =str(key) )
        df['result'][index] =  'any'

        if df['result'][index] == '–––':
            pass

        elif df['result'][index] == str(list(row['answer'])[i]):
            print('Это правльный ответ')
        
        else:
            print('Попробуйте еще раз')
    print('----------------------')       
print(exercise_type)        

once upon  ___  time there  ___  ___  ___   ___   ___  ___ eet little girl
once upon a time there was a sweet little girl
0
Попробуйте еще раз
----------------------
onc ___ ___ ___  ___ sh ___ ___ ___  ___  ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ h ___ ___ ___ r ___  ___ ___ ___  ___ littl ___ ___ ___  ___ c ___ ___ ___ p ___  ___  ___ ___ ___  ___  ___ ___ ___  ___ of ___ r ___ ___ ___  ___  ___  ___ ___ ___  ___ ___ ___ l ___ ___ ___  ___ ___ ___ t
once she gave her a little cap made of red velvet
0
Попробуйте еще раз
1
Попробуйте еще раз
----------------------
 ___  ___  ___  ___   ___ h ___ m  ___ o your gr ___ ndmo ___ h ___ r
take them to your grandmother
0
Попробуйте еще раз
----------------------
she is sick an ___  weak an ___  they will  ___  ___  her well
she is sick and weak and they will do her well
0
Попробуйте еще раз
----------------------
 ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ ___ ___  ___ your ___  ___ ___ ___ a ___ ___ ___  ___ ___ ___  ___ rs _

/var/folders/3s/0vl43zsj2n79hlj09xb7sby40000gn/T/ipykernel_2703/4258132866.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result'][index] =  'any'
